In [1]:
import networkx as nx
import numpy as np
import os.path as op
import scipy as sp

import os
import csv

from collections import OrderedDict
from sklearn.preprocessing import normalize
from sklearn.neighbors import DistanceMetric
from scipy.linalg import svd
from scipy.linalg import norm

np.random.seed(12345678)  # for reproducibility, set random seed

In [9]:
# path = '/Users/gkiar/code/ocp/ndmg-paper/data/new/'
# dsets = ['BNU1', 'HNU1', 'KKI2009', 'MRN114', 'MRN1313', 'NKI1', 'SWU4']
path = '/Users/gkiar/code/ocp/ndmg-paper/data/cloud/'
# dsets = ['BNU1', 'BNU3', 'HNU1', 'Jung2015', 'KKI2009', 'MRN114', 'MRN1313', 'NKI1', 'NKIENH', 'SWU4']
dsets = ['BNU1', 'KKI2009', 'NKI1', 'SWU4', 'HNU1']
# dsets = ['BNU1', 'BNU3', 'HNU1', 'Jung2015', 'MRN114', 'MRN1313', 'NKI1', 'NKIENH', 'SWU4']

# dsets = ['BNU1', 'KKI2009', 'HNU1', 'NKI1', 'SWU4']
# dsets = ['BNU1', 'HNU1', 'KKI2009', 'MRN114', 'MRN1313', 'NKI24', 'SWU4']
# dsets = ['KKI2009']
# dsets = ['SWU4']

dir_names = [path + '/' + d for d in dsets]

N = 70

fs = OrderedDict()
for idx, dd in enumerate(dsets):
    fs[dd] = [root + "/" + fl for root, dirs, files in os.walk(dir_names[idx])
              for fl in files if fl.endswith(".gpickle")]

# ps = {os.path.splitext(os.path.basename(fl))[0] : root + "/" + fl
#       for root, dirs, files in os.walk(path+'phenotypes')
#       for fl in files if fl.endswith(".csv") }

print "Datasets: " + ", ".join([fkey + ' (' + str(len(fs[fkey])) + ')'
                                for fkey in fs])
S = sum([len(fs[key]) for key in fs])
print "Total Subjects: %d" % (S)

Datasets: BNU1 (114), KKI2009 (42), NKI1 (40), SWU4 (454), HNU1 (300)
Total Subjects: 950


In [10]:
def loadGraphs(filenames, verb=False):
    """
    Given a list of files, returns a dictionary of graphs

    Required parameters:
        filenames:
            - List of filenames for graphs
    Optional parameters:
        verb:
            - Toggles verbose output statements
    """
    #  Initializes empty dictionary
    gstruct = OrderedDict()
    for idx, files in enumerate(filenames):
        if verb:
            print "Loading: " + files
        #  Adds graphs to dictionary with key being filename
        fname = os.path.basename(files)
        gstruct[fname] = nx.read_gpickle(files)
    return gstruct

def constructGraphDict(names, fs, verb=False):
    """
    Given a set of files and a directory to put things, loads graphs.

    Required parameters:
        names:
            - List of names of the datasets
        fs:
            - Dictionary of lists of files in each dataset
    Optional parameters:
        verb:
            - Toggles verbose output statements
    """
    #  Loads graphs into memory for all datasets
    graphs = OrderedDict()
    for idx, name in enumerate(names):
        if verb:
            print "Loading Dataset: " + name
        # The key for the dictionary of graphs is the dataset name
        graphs[name] = loadGraphs(fs[name], verb=verb)
    return graphs

def partial_disc(D, labels, subject, trial1, trial2):
    enum = np.arange(D.shape[0])
    idx1 = [i for i, x in enumerate(labels) if x == subject]
    t1 = enum[idx1][trial1]
    t2 = enum[idx1][trial2]
    d_t1_t2 = D[t1][t2]
    
    idx2 = [i for i, x in enumerate(labels) if x != subject]
    d_ra = [D[t1][x] for x in enum[idx2]]
    
    return np.mean(d_t1_t2 < d_ra)

def distance_matrix(data, metric, symmetric = True):
    n = data.shape[2]
    dist_matrix = np.zeros((n, n))
    if symmetric:
        for i in range(n):
            for j in range(i):
                tmp = metric(data[:,:,i] - data[:,:,j])
                dist_matrix[i][j] = tmp
                dist_matrix[j][i] = tmp
    else:
        for i in range(n):
            for j in range(n):
                dist_matrix[i][j] = metric(data[i] - data[j])
    return dist_matrix

def discriminibility(data, labels, metric):
    dist_matrix = distance_matrix(data, metric)
    partials = []
    for s in list(set(labels)):
        num = ids.count(s)
        for t in range(num):
            for tt in range(num):
                if tt != t:
                    p = partial_disc(dist_matrix, labels, s, t, tt)
                    partials.append(p)
    return dist_matrix, np.mean(partials)

In [11]:
graphs = constructGraphDict(dsets, fs, verb=False)

## All subjs, ID = subj id

In [28]:
bigmat = np.zeros((70, 70, 0))
bigids = []

for dset in graphs.keys():
    mat = np.zeros((N, N, len(graphs[dset].keys())))
    ids = []
    c = 0
    for subj in graphs[dset].keys():
        ids += [subj.split("_")[0].split("-")[1]]
        
        tmpg = np.array(nx.adj_matrix(graphs[dset][subj]).todense())
        mat[:, :, c] = tmpg
        c += 1
    print "Shape: {}, {}".format(mat.shape, len(ids))
    dist, disc = discriminibility(mat, ids, norm)
    print '{} ({}): {}'.format(dset, len(graphs[dset].keys()), disc)
    bigmat = np.append(bigmat, mat, axis=2)
    bigids += ids

print "Shape: {}, {}".format(bigmat.shape, len(bigids))
dist, disc = discriminibility(bigmat, bigids, norm)
print 'Total ({}): {}'.format(S, disc)

Shape: (70, 70, 114), 114
BNU1 (114): 0.984257518797
Shape: (70, 70, 42), 42
KKI2009 (42): 1.0
Shape: (70, 70, 40), 40
NKI1 (40): 0.983552631579
Shape: (70, 70, 454), 454
SWU4 (454): 0.883630267826
Shape: (70, 70, 300), 300
HNU1 (300): 0.993232439336
Shape: (70, 70, 950), 950
Total (950): 0.993626477541


In [42]:
bigmat = np.zeros((70, 70, 0))
bigids = []

for dset in graphs.keys():
    mat = np.zeros((N, N, len(graphs[dset].keys())))
    ids = []
    c = 0
    for subj in graphs[dset].keys():
        
        tmpg = np.array(nx.adj_matrix(graphs[dset][subj]).todense())
        ratio = np.sum(tmpg > 0)/1.0/N/N
        nedges = np.sum(tmpg)
#         print nedges
        if ratio < 0.32 or ratio > 0.54 or nedges > 8e7:
            print np.sum(tmpg > 0)
            mat = np.delete(mat, c, 2)
        else:
            ids += [subj.split("_")[0].split("-")[1]]
            mat[:, :, c] = tmpg
            c += 1
    dist, disc = discriminibility(mat, ids, norm)
    print '{} ({}): {}'.format(dset, len(graphs[dset].keys()), disc)
    bigmat = np.append(bigmat, mat, axis=2)
    bigids += ids

dist, disc = discriminibility(bigmat, bigids, norm)
print 'Total ({}): {}'.format(len(bigids), disc)

1110
BNU1 (114): 0.990910553411
KKI2009 (42): 1.0
NKI1 (40): 0.983552631579
1532
2804
1510
1506
SWU4 (454): 0.893292560858
2692
2702
2650
HNU1 (300): 0.993336485986
Total (942): 0.993773328929
